# 70 Audit Meta DB (KPI確認 & テーブル総点検)
このノートブックは、`forecast_meta.db`（SQLite）に対して以下を段階的に確認します。
- DB内のテーブル一覧・行数（row count）
- 各テーブルのカラム定義（schema）
- **データが入っているテーブルだけ**中身をプレビュー
- KPI（今回の要件）として重要なチェック（ゲーム定義/スロット/7モデルセレクタ/レジストリ種）
- （任意）`neuralforecast` がインストールされている場合のシグネチャ（signature）簡易走査

注意: 現状のDBは「雛形 + シード（seed）」中心です。**本格的なライブラリ自動走査の投入は次段**で行う設計になっています。

In [1]:
# セットアップ
from __future__ import annotations

import os
import json
import sqlite3
from pathlib import Path
import pandas as pd

from IPython.display import display

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 160)

## 1) プロジェクトルートとDBファイルの自動検出
`pyproject.toml` を起点にプロジェクトルートを推定し、DB候補を探します。

In [2]:
def find_project_root(start: Path | None = None) -> Path:
    p = (start or Path.cwd()).resolve()
    for _ in range(10):
        if (p / "pyproject.toml").exists():
            return p
        if p.parent == p:
            break
        p = p.parent
    return (start or Path.cwd()).resolve()

root = find_project_root()
root

WindowsPath('C:/fcast')

In [3]:
def find_db_file(root: Path) -> Path:
    candidates = [
        root / "forecast_meta.db",
        root / "data" / "forecast_meta.db",
        root / "data" / "meta" / "forecast_meta.db",
        root / "meta" / "forecast_meta.db",
    ]
    for c in candidates:
        if c.exists():
            return c

    # 最終手段: それっぽい *.db を探索
    dbs = list(root.rglob("*.db"))
    if not dbs:
        raise FileNotFoundError("*.db が見つかりません。`forecast_meta.db` を配置するか、パスを指定してください。")

    meta_dbs = [p for p in dbs if "meta" in p.name.lower() or "forecast" in p.name.lower()]
    return (meta_dbs or dbs)[0]

db_path = find_db_file(root)
db_path

WindowsPath('C:/fcast/notebooks/forecast_meta.db')

## 2) テーブル一覧・行数（row count）

In [4]:
conn = sqlite3.connect(str(db_path))

def list_tables(conn: sqlite3.Connection) -> list[str]:
    q = '''
    SELECT name
    FROM sqlite_master
    WHERE type='table' AND name NOT LIKE 'sqlite_%'
    ORDER BY name
    '''
    return [r[0] for r in conn.execute(q).fetchall()]

tables = list_tables(conn)
len(tables), tables[:10]

(48,
 ['datasets_artifact',
  'datasets_dataset',
  'datasets_game',
  'datasets_game_slot',
  'datasets_series',
  'datasets_version',
  'features_availability',
  'features_def',
  'features_generator',
  'features_materialization'])

In [5]:
def table_row_counts(conn: sqlite3.Connection, tables: list[str]) -> pd.DataFrame:
    rows = []
    for t in tables:
        n = conn.execute(f"SELECT COUNT(*) FROM {t}").fetchone()[0]
        rows.append({"table": t, "rows": n})
    df = pd.DataFrame(rows).sort_values(["rows", "table"], ascending=[False, True]).reset_index(drop=True)
    return df

df_counts = table_row_counts(conn, tables)
display(df_counts)
display(df_counts[df_counts["rows"] > 0])

,table,rows
0,datasets_game_slot,30
1,models_series_selector,7
2,datasets_game,6
3,registry_param_enum,5
4,registry_library,2
5,registry_release,2
6,registry_symbol,2
7,datasets_dataset,1
8,datasets_version,1
9,features_set,1


,table,rows
0,datasets_game_slot,30
1,models_series_selector,7
2,datasets_game,6
3,registry_param_enum,5
4,registry_library,2
5,registry_release,2
6,registry_symbol,2
7,datasets_dataset,1
8,datasets_version,1
9,features_set,1


## 3) テーブル定義（schema: カラム名・型・PKなど）
SQLiteでは `PRAGMA table_info` を使ってカラム定義を取得できます。

In [6]:
def table_schema(conn: sqlite3.Connection, table: str) -> pd.DataFrame:
    df = pd.read_sql_query(f"PRAGMA table_info('{table}')", conn)
    df.insert(0, "table", table)
    return df

schemas = pd.concat([table_schema(conn, t) for t in tables], ignore_index=True)
display(schemas.head(50))

,table,cid,name,type,notnull,dflt_value,pk
0,datasets_artifact,0,artifact_id,VARCHAR(36),1,None,1
1,datasets_artifact,1,dataset_version_id,VARCHAR(36),1,None,0
2,datasets_artifact,2,layer,VARCHAR,1,None,0
3,datasets_artifact,3,table_name,VARCHAR,1,None,0
4,datasets_artifact,4,storage_uri,TEXT,1,None,0
5,datasets_artifact,5,schema_hash,VARCHAR,0,None,0
6,datasets_artifact,6,created_at,DATETIME,1,None,0
7,datasets_dataset,0,dataset_id,VARCHAR(36),1,None,1
8,datasets_dataset,1,name,VARCHAR,1,None,0
9,datasets_dataset,2,game_id,VARCHAR(36),1,None,0


### 3-2) 外部キー（FK: Foreign Key）とインデックスの確認
ER（関係）を点検したいときに有効です。SQLiteでは `PRAGMA foreign_key_list` / `PRAGMA index_list` を使います。

In [7]:
def foreign_keys(conn: sqlite3.Connection, table: str) -> pd.DataFrame:
    return pd.read_sql_query(f"PRAGMA foreign_key_list('{table}')", conn)

def index_list(conn: sqlite3.Connection, table: str) -> pd.DataFrame:
    return pd.read_sql_query(f"PRAGMA index_list('{table}')", conn)

# 例: datasets_game_slot のFKとインデックス
display(foreign_keys(conn, "datasets_game_slot"))
display(index_list(conn, "datasets_game_slot"))

,id,seq,table,from,to,on_update,on_delete,match
0,0,0,datasets_game,game_id,game_id,NO ACTION,NO ACTION,NONE


,seq,name,unique,origin,partial
0,0,sqlite_autoindex_datasets_game_slot_2,1,u,0
1,1,sqlite_autoindex_datasets_game_slot_1,1,pk,0


### 3-1) “テーブルごとのカラム一覧”をすぐ見たい場合

In [8]:
def schema_by_table(schemas: pd.DataFrame, table: str) -> pd.DataFrame:
    return schemas[schemas["table"] == table].sort_values("cid").reset_index(drop=True)

# 例: models_series_selector のschema
display(schema_by_table(schemas, "models_series_selector"))

,table,cid,name,type,notnull,dflt_value,pk
0,models_series_selector,0,selector_id,VARCHAR(36),1,None,1
1,models_series_selector,1,name,VARCHAR,1,None,0
2,models_series_selector,2,game_code,VARCHAR,1,None,0
3,models_series_selector,3,unique_id_list,JSON,1,None,0
4,models_series_selector,4,created_at,DATETIME,1,None,0


## 4) 中身プレビュー（データが入っているテーブルのみ）
現時点で `rows>0` のテーブルだけを上から順にプレビューします。

In [9]:
def head_table(conn: sqlite3.Connection, table: str, n: int = 20) -> pd.DataFrame:
    return pd.read_sql_query(f"SELECT * FROM {table} LIMIT {n}", conn)

non_empty = df_counts[df_counts["rows"] > 0]["table"].tolist()
non_empty

['datasets_game_slot',
 'models_series_selector',
 'datasets_game',
 'registry_param_enum',
 'registry_library',
 'registry_release',
 'registry_symbol',
 'datasets_dataset',
 'datasets_version',
 'features_set',
 'registry_capability',
 'registry_param',
 'registry_param_constraint',
 'registry_signature',
 'registry_symbol_capability']

In [10]:
for t in non_empty:
    print("="*120)
    print(f"[{t}] rows={int(df_counts.loc[df_counts.table==t,'rows'].iloc[0])}")
    display(schema_by_table(schemas, t))
    display(head_table(conn, t, n=20))

[datasets_game_slot] rows=30


,table,cid,name,type,notnull,dflt_value,pk
0,datasets_game_slot,0,game_slot_id,VARCHAR(36),1,None,1
1,datasets_game_slot,1,game_id,VARCHAR(36),1,None,0
2,datasets_game_slot,2,slot_code,VARCHAR,1,None,0
3,datasets_game_slot,3,slot_index,INTEGER,1,None,0
4,datasets_game_slot,4,slot_role,VARCHAR,1,None,0
5,datasets_game_slot,5,slot_meta,JSON,1,None,0


,game_slot_id,game_id,slot_code,slot_index,slot_role,slot_meta
0,a1e5bbbd-5cbb-4df8-b054-edad8bd795ae,6a09f0b3-cc95-4b20-8b2d-8c0822e6ed43,N1,1,main,{}
1,c36b0b38-3d1d-4f54-b966-2d4a6e265222,6a09f0b3-cc95-4b20-8b2d-8c0822e6ed43,N2,2,main,{}
2,d4e82f32-ec11-4f13-baf7-331d135cecda,6a09f0b3-cc95-4b20-8b2d-8c0822e6ed43,N3,3,main,{}
3,1a0b607c-3a6f-48f0-8b4f-c5b8e8f82aa4,946ae1eb-d989-40ab-93e1-982e827cd150,N1,1,main,{}
4,446b77b4-cb69-4d25-9d95-75036c9b6af6,946ae1eb-d989-40ab-93e1-982e827cd150,N2,2,main,{}
5,9667e042-805d-4385-b6b3-985c4ebc8332,946ae1eb-d989-40ab-93e1-982e827cd150,N3,3,main,{}
6,5d453318-34c0-461d-a154-607e80f69d85,946ae1eb-d989-40ab-93e1-982e827cd150,N4,4,main,{}
7,052270e3-2be4-492a-97cb-53d98eacd797,d6b20851-c15e-4bea-a5ca-13e5518d17c7,N1,1,main,{}
8,fa4e0d01-a07c-438b-9ae4-bd18f722827f,d6b20851-c15e-4bea-a5ca-13e5518d17c7,N2,2,main,{}
9,6f1d10b7-ad1e-4743-890a-84ab01e92a0d,d6b20851-c15e-4bea-a5ca-13e5518d17c7,N3,3,main,{}


[models_series_selector] rows=7


,table,cid,name,type,notnull,dflt_value,pk
0,models_series_selector,0,selector_id,VARCHAR(36),1,None,1
1,models_series_selector,1,name,VARCHAR,1,None,0
2,models_series_selector,2,game_code,VARCHAR,1,None,0
3,models_series_selector,3,unique_id_list,JSON,1,None,0
4,models_series_selector,4,created_at,DATETIME,1,None,0


...

,table,cid,name,type,notnull,dflt_value,pk
0,registry_signature,0,signature_id,VARCHAR(36),1,None,1
1,registry_signature,1,symbol_id,VARCHAR(36),1,None,0
2,registry_signature,2,signature_text,TEXT,1,None,0
3,registry_signature,3,return_type_repr,TEXT,0,None,0
4,registry_signature,4,source,VARCHAR,1,None,0
5,registry_signature,5,extracted_at,DATETIME,1,None,0


,signature_id,symbol_id,signature_text,return_type_repr,source,extracted_at
0,54802009-d302-4eaa-94d2-e153d89b601d,c91e3a0d-0169-41b1-99a8-0843a6b98953,"fit(df=None, static_df=None, val_size=0, use_i...",self,manual,2026-01-29 07:17:45.346970


[registry_symbol_capability] rows=1


,table,cid,name,type,notnull,dflt_value,pk
0,registry_symbol_capability,0,symbol_capability_id,VARCHAR(36),1,None,1
1,registry_symbol_capability,1,symbol_id,VARCHAR(36),1,None,0
2,registry_symbol_capability,2,capability_id,VARCHAR(36),1,None,0
3,registry_symbol_capability,3,value_json,JSON,1,None,0
4,registry_symbol_capability,4,source,VARCHAR,0,None,0
5,registry_symbol_capability,5,created_at,DATETIME,1,None,0


,symbol_capability_id,symbol_id,capability_id,value_json,source,created_at
0,d0802b80-329b-4e8b-a00b-6bd4f18c3019,c91e3a0d-0169-41b1-99a8-0843a6b98953,567d4ec3-98de-4b93-a40a-4468dc44b534,"{""enabled"": true}",manual,2026-01-29 07:17:45.360753


## 5) KPIチェック（今回の要件に対する最低限の整合性）
ここは“現状のDBが設計どおりに初期化/シードできているか”を確認します。
- 宝くじゲーム定義（numbers3/4, miniloto, loto6/7, bingo5）
- ゲームごとの桁（slot）の個数
- LOTO6の **各桁6モデル + 全桁1モデル = 7モデル** セレクタ
- レジストリ（library/release/symbol/signature/param）が最低限存在

In [11]:
games = pd.read_sql_query("SELECT * FROM datasets_game", conn)
display(games[["game_code","rules_json","created_at"]])

expected_games = {"numbers3","numbers4","miniloto","loto6","loto7","bingo5"}
found_games = set(games["game_code"].tolist())
print("games OK:", expected_games.issubset(found_games), "missing:", sorted(expected_games - found_games))

,game_code,rules_json,created_at
0,numbers3,"{""k"": 3, ""maxv"": 9}",2026-01-29 07:18:14.132462
1,numbers4,"{""k"": 4, ""maxv"": 9}",2026-01-29 07:18:14.135015
2,miniloto,"{""k"": 5, ""maxv"": 31}",2026-01-29 07:18:14.137906
3,loto6,"{""k"": 6, ""maxv"": 43}",2026-01-29 07:18:14.139927
4,loto7,"{""k"": 7, ""maxv"": 37}",2026-01-29 07:18:14.141458
5,bingo5,"{""k"": 5, ""maxv"": 39}",2026-01-29 07:18:14.142827


games OK: True missing: []


In [12]:
# ゲームごとの slot 個数が rules_json.k と一致するか（簡易チェック）
slots = pd.read_sql_query(
    '''
    SELECT g.game_code, s.slot_code, s.slot_index, s.slot_role
    FROM datasets_game_slot s
    JOIN datasets_game g ON g.game_id = s.game_id
    ''',
    conn,
)

def parse_k(rules_json: str) -> int | None:
    try:
        return int(json.loads(rules_json).get("k"))
    except Exception:
        return None

game_k = games.assign(k=games["rules_json"].map(parse_k))[["game_code","k"]]
slot_counts = slots.groupby("game_code").size().reset_index(name="slot_n")
chk = game_k.merge(slot_counts, on="game_code", how="left")
chk["ok"] = chk["k"] == chk["slot_n"]
display(chk.sort_values("game_code"))
print("slot count OK:", bool(chk["ok"].all()))

,game_code,k,slot_n,ok
5,bingo5,5,5,True
3,loto6,6,6,True
4,loto7,7,7,True
2,miniloto,5,5,True
0,numbers3,3,3,True
1,numbers4,4,4,True


slot count OK: True


In [13]:
# LOTO6: 6モデル + 全桁1モデル = 7モデルの存在チェック
selectors = pd.read_sql_query(
    "SELECT * FROM models_series_selector WHERE game_code='loto6' ORDER BY name",
    conn,
)
display(selectors)

ok7 = len(selectors) == 7
print("7 selectors OK:", ok7, "count:", len(selectors))

# 内訳チェック（N1..N6 + ALL）
needed = {f"loto6_N{i}" for i in range(1,7)} | {"loto6_ALL"}
found = set(selectors["name"].tolist())
print("selector names OK:", needed.issubset(found), "missing:", sorted(needed - found))

,selector_id,name,game_code,unique_id_list,created_at
0,bfe846bd-45e2-4698-bbfd-41bc051d00d1,loto6_ALL,loto6,"[""N1"", ""N2"", ""N3"", ""N4"", ""N5"", ""N6""]",2026-01-29 07:25:36.356708
1,806b8adc-83f7-451b-8501-4bd90c18e4d4,loto6_N1,loto6,"[""N1""]",2026-01-29 07:25:36.356662
2,d636154e-821b-4f24-b820-3713464a6f57,loto6_N2,loto6,"[""N2""]",2026-01-29 07:25:36.356673
3,c5c79248-8471-4ef0-a211-6283624369e3,loto6_N3,loto6,"[""N3""]",2026-01-29 07:25:36.356680
4,48d9e327-35ca-49e1-80b0-31de27e6ef2b,loto6_N4,loto6,"[""N4""]",2026-01-29 07:25:36.356687
5,82b2a128-9583-471b-90d3-f3ef3bdaabc0,loto6_N5,loto6,"[""N5""]",2026-01-29 07:25:36.356696
6,72d9756f-d1b5-48c5-a124-e7d566806ea9,loto6_N6,loto6,"[""N6""]",2026-01-29 07:25:36.356702


7 selectors OK: True count: 7
selector names OK: True missing: []


In [14]:
# レジストリ最小チェック（現状はseed中心）
reg_counts = df_counts[df_counts["table"].str.startswith("registry_")].copy()
display(reg_counts)

must_have = ["registry_library","registry_release","registry_symbol","registry_signature"]
missing = [t for t in must_have if t not in tables]
print("registry tables exist:", not missing, "missing:", missing)

for t in must_have:
    n = int(df_counts.loc[df_counts.table==t, "rows"].iloc[0]) if t in tables else 0
    print(f"{t}: rows={n}")

,table,rows
3,registry_param_enum,5
4,registry_library,2
5,registry_release,2
6,registry_symbol,2
10,registry_capability,1
11,registry_param,1
12,registry_param_constraint,1
13,registry_signature,1
14,registry_symbol_capability,1
32,registry_loss,0


registry tables exist: True missing: []
registry_library: rows=2
registry_release: rows=2
registry_symbol: rows=2
registry_signature: rows=1


## 6) （任意）neuralforecast の簡易走査（signatureだけ）
現時点のDBは `20_seed_registry.ipynb` により **手動で最低限のレコードを投入**しています。
ここでは `neuralforecast` がインストールされている場合に、`inspect.signature` で
メソッドの引数を覗き、DBの `registry_signature` と比較する“軽いファクトチェック”を行います。

※ 大量のモデル・loss・capabilities を本格走査してDBに投入するのは次段の実装（スキャナ作成）で行います。

In [15]:
import inspect

def try_scan_neuralforecast():
    try:
        import neuralforecast
        from neuralforecast.core import NeuralForecast
    except Exception as e:
        print("neuralforecast import failed (未インストール or 依存関係):", repr(e))
        return None

    sig_fit = inspect.signature(NeuralForecast.fit)
    sig_pred = inspect.signature(NeuralForecast.predict)
    sig_cv = inspect.signature(NeuralForecast.cross_validation)

    return {
        "neuralforecast_version": getattr(neuralforecast, "__version__", None),
        "NeuralForecast.fit": str(sig_fit),
        "NeuralForecast.predict": str(sig_pred),
        "NeuralForecast.cross_validation": str(sig_cv),
    }

scan = try_scan_neuralforecast()
scan

neuralforecast import failed (未インストール or 依存関係): ModuleNotFoundError("No module named 'neuralforecast'")


In [16]:
# DB側に保存されている signature（seed）と比較（fitのみ）
if scan is not None:
    db_fit = pd.read_sql_query(
        '''
        SELECT rs.signature_text
        FROM registry_signature rs
        JOIN registry_symbol sym ON sym.symbol_id = rs.symbol_id
        WHERE sym.qualname LIKE '%NeuralForecast.fit%'
        LIMIT 1
        ''',
        conn,
    )

    db_fit_sig = db_fit["signature_text"].iloc[0] if len(db_fit) else None
    print("DB(signature_text):", db_fit_sig)
    print("SCAN(signature):     ", scan.get("NeuralForecast.fit"))
    print("一致:", db_fit_sig == scan.get("NeuralForecast.fit"))
else:
    print("skip")

skip


## 7) 次の一手（スキャナ投入の設計メモ）
- `registry_*` は **“ライブラリを走査して事実を登録する層”**
- `models_* / runs_* / datasets_* / features_*` は **“実運用で回す層”**

このノートでは DB の“現状”を点検するだけなので、
本格的な走査（モデル一覧、loss互換、exogenous対応、fit/predict/cv引数の型/制約）を
自動登録する場合は、`fcast.plugins` にスキャナ（plugin）を追加し、
`registry_*` に投入するバッチ（CLI or notebook）を作るのが筋が良いです。